<a href="https://colab.research.google.com/github/nicholasfurl/Great-Courses/blob/main/Lesson09_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#In this notebook we will create a classifier to predict diabetes using a support vector machine (SVM) 
#varying the number of degrees of the polynomial of the SVM.

In [ ]:
#Again, keeping this here for reference but will try to keep actual imports near 
#their function calls to learn their associations whenever possible

import random
from sklearn import svm
import matplotlib.pyplot as plt

In [1]:
#Download the diabetes data from GitHub. 
#We used this with the first decision tree lesson, I believe.

!wget https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv

--2023-01-06 10:34:05--  https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23873 (23K) [text/plain]
Saving to: ‘diabetes.csv’

diabetes.csv        100%[===================>]  23.31K  --.-KB/s    in 0.001s  

2023-01-06 10:34:05 (19.4 MB/s) - ‘diabetes.csv’ saved [23873/23873]

